In [3]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image


DATA_DIR = 'data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')
SIZE = (224, 224)
BATCH_SIZE = 16



num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

gen = keras.preprocessing.image.ImageDataGenerator()
val_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)

model = keras.applications.resnet50.ResNet50()

classes = list(iter(batches.class_indices))
model.layers.pop()
for layer in model.layers:
    layer.trainable=False
last = model.layers[-1].output
x = Dense(len(classes), activation="softmax")(last)
finetuned_model = Model(model.input, x)
finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
for c in batches.class_indices:
    classes[batches.class_indices[c]] = c
finetuned_model.classes = classes

early_stopping = EarlyStopping(patience=10)
checkpointer = ModelCheckpoint('resnet50_best.h5', verbose=1, save_best_only=True)

finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=1000, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)
finetuned_model.save('resnet50_final.h5')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\train'

In [3]:
import tensorflow as tf


In [4]:
tf.keras


<module 'tensorflow._api.v1.keras' from 'C:\\Users\\Vardan\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v1\\keras\\__init__.py'>

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2681694803144744443
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9218918974
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18208810455755712335
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 0s 463us/step - loss: 0.7271 - acc: 0.5000
Epoch 2/20
1000/1000 [==============================] - 0s 23us/step - loss: 0.7135 - acc: 0.5060
Epoch 3/20
1000/1000 [==============================] - 0s 24us/step - loss: 0.7123 - acc: 0.4900
Epoch 4/20
1000/1000 [==============================] - 0s 23us/step - loss: 0.7059 - acc: 0.4840
Epoch 5/20
1000/1000 [==============================] - 0s 25us/step - loss: 0.7085 - acc: 0.4830
Epoch 6/20
1000/1000 [==============================] - 0s 22us/step - loss: 0.6969 - acc: 0.5310
Epoch 7/20
1000/1000 [==============================] - 0s 24us/step - loss: 0.6974 - acc: 0.5140
Epoch 8/20
1000/1000 [==============================] - 0s 23us/step - loss: 0.6970 - acc: 0.5040
Epoch 9/20
1000/1000 [==============================] - 0s 25us/step - loss: 0.6979 - acc: 0.5130
Epoch 10/20
1000/1000 [==============================] - 0s 23us/step - loss: 0.6936 - acc: 0.5130
Epoch 11/20
1000/1